In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

import spacy
from spacy.tokens import DocBin
from spacy.util import filter_spans
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
d1 = pd.read_excel('../data_source/G1.xlsx')
d1 = d1.drop(columns=['Unnamed: 0'])
d1

,ID,tags,text
0,NCT02105766,"8:16:chronic_disease,20:32:treatment",portal fibrosis by liver biopsy
1,NCT03008070,22:34:treatment,Contra-indication to liver biopsy
2,NCT03008070,",32:44:treatment,,",Have a stable weight since the liver biopsy wa...
3,NCT03008070,"26:38:treatment,",Subject agrees to have a liver biopsy performe...
4,NCT02515708,",43:55:treatment,",Liver steatosis (on visual estimate or on live...
...,...,...,...
7351,NCT00147056,"46:63:cancer,73:80:treatment,82:91:treatment,9...",Subjects with (newly diagnosed or recurrent) m...
7352,NCT00107289,"1:5:treatment,11:13:cancer,39:48:treatment",MIBG-avid NB and evaluable disease on MIBG sca...
7353,NCT00107289,"27:29:cancer,157:171:cancer,186:208:chronic_di...",ust have the diagnosis of NB in accordance wit...
7354,NCT00075387,"40:51:treatment,53:69:treatment,71:90:treatmen...",Subjects who have contraindications to carbopl...


In [3]:
d1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7356 entries, 0 to 7355
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      7356 non-null   object
 1   tags    7356 non-null   object
 2   text    7356 non-null   object
dtypes: object(3)
memory usage: 172.5+ KB


In [4]:
# d1 = d1[~d1.duplicated()]

In [5]:
# tags = d1['tags'].tolist()
# text = d1['text'].tolist()
# tags = [[y for y in x.split(',') if len(y)!=0] for x in tags]

# # tags
# print(len(tags), len(text))
# idx_to_remove = []
# for i, x in enumerate(tags):
#     mx_idx = -1
#     for y in x:
#         start_idx = int(y.split(':')[0])
#         end_idx = int(y.split(':')[1])
#         mx_idx = max(mx_idx, start_idx, end_idx)

#         if(start_idx>end_idx):
#             print(y.split(':'))
#             print(i,x,"====", start_idx, end_idx)
#             idx_to_remove.append(i)
        
#         if(mx_idx > len(text[i])+1):
#             print(i, x, text[i])
#             idx_to_remove.append(i)

In [6]:
def idxs_to_remove(tags_list, text_list):
    idx_to_remove = []

    for i, x in enumerate(tags_list):
        mx_idx = -1
        for y in x:
            start_idx = int(y.split(':')[0])
            end_idx = int(y.split(':')[1])
            mx_idx = max(mx_idx, start_idx, end_idx)

            if(start_idx>end_idx):
                # print(y.split(':'))
                # print(i,x,"====", start_idx, end_idx)
                idx_to_remove.append(i)
            
            if(mx_idx > len(text_list[i])+1):
                # print(i, x, text_list[i])
                idx_to_remove.append(i)

    return idx_to_remove

In [7]:
def quick_eda(df):
    print(df.info(), "\n")
    print("No. of duplicates in dataframe - {}".format(df.loc[df.duplicated()].shape[0]))
    
    tags = df['tags'].tolist()
    text = df['text'].tolist()
    tags = [[y for y in x.split(',') if len(y)!=0] for x in tags]
    set_labels = set([y.split(':')[-1] for x in tags for y in x])
    print("Unique labels in tags columns: {}".format(set_labels),"\n\n")

    return 

quick_eda(d1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7356 entries, 0 to 7355
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      7356 non-null   object
 1   tags    7356 non-null   object
 2   text    7356 non-null   object
dtypes: object(3)
memory usage: 172.5+ KB
None 

No. of duplicates in dataframe - 167
Unique labels in tags columns: {'chronic_disease', 'cancer', 'treatment', 'allergy_name'} 




In [8]:
def create_entites(tags, text):
    entities = []
    for x in tags:
        temp = []
        for y in x:
            temp_ls = y.split(':')
            start_idx = int(temp_ls[0])
            end_idx = int(temp_ls[1])
            label = temp_ls[-1]
            tup = (start_idx, end_idx, label)
            # print(tup)
            temp.append(tup)
        entities.append(temp)
    
    return entities

In [9]:
def preprocess(df):
    print('original_shape: {}'.format(df.shape))
    df = df.drop_duplicates()
    print('shape after duplicates drop: {}'.format(df.shape))
    tags = df['tags'].tolist()
    text = df['text'].tolist()
    tags = [[y for y in x.split(',') if len(y)!=0] for x in tags]
    # print(tags[:5])
    entities = create_entites(tags=tags, text=text)
    df['entities'] = entities
    remove_idx_ls = idxs_to_remove(tags, text)
    print("{} indices to remove".format(len(remove_idx_ls)))
    print('Removing_index: {}'.format(remove_idx_ls))
    df = df.drop(remove_idx_ls, axis=0).reset_index()

    print("Final columns: {}".format(df.columns))
    return df


In [10]:
d1 = preprocess(d1)

d1.shape

original_shape: (7356, 3)
shape after duplicates drop: (7189, 3)
2 indices to remove
Removing_index: [1522, 4236]
Final columns: Index(['index', 'ID', 'tags', 'text', 'entities'], dtype='object')


(7187, 5)

In [11]:
dfs = []
for f in glob('../data_source/*.xlsx'):
    print(f,"\n")
    df = pd.read_excel(f)
    df = df.drop(columns=['Unnamed: 0'])
    df = df.dropna(axis=0)
    quick_eda(df)
    df = preprocess(df)

    dfs.append(df)
    print("\n\n-------------------------------------------------------------------------")
    

../data_source/G1.xlsx 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7356 entries, 0 to 7355
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      7356 non-null   object
 1   tags    7356 non-null   object
 2   text    7356 non-null   object
dtypes: object(3)
memory usage: 172.5+ KB
None 

No. of duplicates in dataframe - 167
Unique labels in tags columns: {'chronic_disease', 'cancer', 'treatment', 'allergy_name'} 


original_shape: (7356, 3)
shape after duplicates drop: (7189, 3)
2 indices to remove
Removing_index: [1522, 4236]
Final columns: Index(['index', 'ID', 'tags', 'text', 'entities'], dtype='object')


-------------------------------------------------------------------------
../data_source/G3.xlsx 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6277 entries, 0 to 6276
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      6277 non-null   object

In [12]:
df1, df2, df3 = dfs[0], dfs[1], dfs[2]

In [13]:
df1

,index,ID,tags,text,entities
0,0,NCT02105766,"8:16:chronic_disease,20:32:treatment",portal fibrosis by liver biopsy,"[(8, 16, chronic_disease), (20, 32, treatment)]"
1,1,NCT03008070,22:34:treatment,Contra-indication to liver biopsy,"[(22, 34, treatment)]"
2,2,NCT03008070,",32:44:treatment,,",Have a stable weight since the liver biopsy wa...,"[(32, 44, treatment)]"
3,3,NCT03008070,"26:38:treatment,",Subject agrees to have a liver biopsy performe...,"[(26, 38, treatment)]"
4,4,NCT02515708,",43:55:treatment,",Liver steatosis (on visual estimate or on live...,"[(43, 55, treatment)]"
...,...,...,...,...,...
7182,7351,NCT00147056,"46:63:cancer,73:80:treatment,82:91:treatment,9...",Subjects with (newly diagnosed or recurrent) m...,"[(46, 63, cancer), (73, 80, treatment), (82, 9..."
7183,7352,NCT00107289,"1:5:treatment,11:13:cancer,39:48:treatment",MIBG-avid NB and evaluable disease on MIBG sca...,"[(1, 5, treatment), (11, 13, cancer), (39, 48,..."
7184,7353,NCT00107289,"27:29:cancer,157:171:cancer,186:208:chronic_di...",ust have the diagnosis of NB in accordance wit...,"[(27, 29, cancer), (157, 171, cancer), (186, 2..."
7185,7354,NCT00075387,"40:51:treatment,53:69:treatment,71:90:treatmen...",Subjects who have contraindications to carbopl...,"[(40, 51, treatment), (53, 69, treatment), (71..."


In [14]:
training_data = {'classes' : ['allergy_name', 'treatment', 'chronic_disease', 'cancer'],
                 'annotations': []}

for entities, text in zip(df1['entities'], df1['text']):
    temp_dict = {'entities': entities,
                 'text' : text}
    training_data['annotations'].append(temp_dict)

In [15]:
training_data['annotations'][0]

{'entities': [(8, 16, 'chronic_disease'), (20, 32, 'treatment')],
 'text': 'portal fibrosis by liver biopsy'}

In [16]:
nlp = spacy.blank('en')
doc_bin = DocBin()

In [17]:
for training_example  in tqdm(training_data['annotations'][:100]): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)


model_file_dir = "../model_file_dir"

if not os.path.exists(model_file_dir):
    os.makedirs(model_file_dir)

doc_bin.to_disk ("training_data100.spacy")

100%|██████████| 100/100 [00:00<00:00, 6243.75it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


In [18]:
%%bash
python -m spacy init fill-config base_config.cfg config.cfg

/Users/raunakanand/Documents/Work_R/interview_problems/.venv/lib/python3.12/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/raunakanand/Documents/Work_R/interview_problems/.venv/lib/python3.12/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [19]:
# mx = -1
# for x in training_data['annotations']:
#     mx = max(mx, len(x['text']))

# mx

In [20]:
# torch.mps.set_per_process_memory_fraction(0.7)

In [51]:
%%bash
python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy --gpu-id 0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================


/Users/raunakanand/Documents/Work_R/interview_problems/.venv/lib/python3.12/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/raunakanand/Documents/Work_R/interview_problems/.venv/lib/python3.12/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/raunakanand/Documents/Work_R/interview_problems/.venv/lib/python3.12/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initializ

✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'ner']
ℹ Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  -------------  --------  ------  ------  ------  ------
  0       0         636.77    789.00    3.20    1.70   26.15    0.03
⚠ Aborting and saving the final best model. Encountered exception:
RuntimeError('MPS backend out of memory (MPS allocated: 5.27 GB, other
allocations: 31.05 GB, max allowed: 36.27 GB). Tried to allocate 1024 bytes on
private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit
for memory allocations (may cause system failure).')


Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/raunakanand/Documents/Work_R/interview_problems/.venv/lib/python3.12/site-packages/spacy/__main__.py", line 4, in <module>
    setup_cli()
  File "/Users/raunakanand/Documents/Work_R/interview_problems/.venv/lib/python3.12/site-packages/spacy/cli/_util.py", line 87, in setup_cli
    command(prog_name=COMMAND)
  File "/Users/raunakanand/Documents/Work_R/interview_problems/.venv/lib/python3.12/site-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/raunakanand/Documents/Work_R/interview_problems/.venv/lib/python3.12/site-packages/typer/core.py", line 783, in main
    return _main(
           ^^^^^^
  File "/Users/raunakanand/Documents/Work_R/interview_problems/.venv/lib/python3.12/site-packages/typer/core.py", line 225, in _main
    rv = self.invoke

CalledProcessError: Command 'b'python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy --gpu-id 0\n'' returned non-zero exit status 1.

In [45]:
nlp_ner = spacy.load('model-best')

In [46]:
doc = nlp_ner(training_data['annotations'][101]['text'])
doc

HIV Positive

In [47]:
print(training_data['annotations'][101]['text'])
print(training_data['annotations'][101]['entities'])

HIV Positive
[(1, 4, 'chronic_disease')]


In [48]:
colors = {"chronic_disease": "#F67DE3", "MEDICINE": "#7DF6D9", "MEDICALCONDITION":"#FFFFFF"}
options = {"colors": colors} 
spacy.displacy.render(doc, style="ent", jupyter=True,options= options)

In [49]:
for ent in doc.ents:
    print(ent.text, ent.label_)

In [50]:
doc.ents

()

In [ ]:
# nlp = spacy.load("en_core_web_sm")
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")